In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
goog = yf.Ticker("goog")

In [ ]:
df = goog.history(period="max")

In [ ]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2004-08-19,49.813290,51.835709,47.800831,49.982655,44871361,0,0.0
2004-08-20,50.316402,54.336334,50.062355,53.952770,22942874,0,0.0
2004-08-23,55.168217,56.528118,54.321388,54.495735,18342897,0,0.0
2004-08-24,55.412300,55.591629,51.591621,52.239197,15319808,0,0.0
2004-08-25,52.284027,53.798351,51.746044,52.802086,9232276,0,0.0
...,...,...,...,...,...,...,...
2021-07-06,2588.989990,2597.689941,2568.179932,2595.419922,1067500,0,0.0
2021-07-07,2606.820068,2612.798096,2595.199951,2601.550049,834000,0,0.0
2021-07-08,2565.000000,2600.649902,2560.800049,2583.540039,989000,0,0.0


In [ ]:
df["recommend"] = 0

In [ ]:
days = 3
for i in range(4252-days):
  if (df.iloc[i+1:i+1+days,3].mean() > df.iat[i,3]):
    df.iloc[i,7] = 1


In [ ]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,recommend
Date,,,,,,,,
2004-08-19,49.813290,51.835709,47.800831,49.982655,44871361,0,0.0,1
2004-08-20,50.316402,54.336334,50.062355,53.952770,22942874,0,0.0,0
2004-08-23,55.168217,56.528118,54.321388,54.495735,18342897,0,0.0,0
2004-08-24,55.412300,55.591629,51.591621,52.239197,15319808,0,0.0,1
2004-08-25,52.284027,53.798351,51.746044,52.802086,9232276,0,0.0,0
...,...,...,...,...,...,...,...,...
2021-07-06,2588.989990,2597.689941,2568.179932,2595.419922,1067500,0,0.0,0
2021-07-07,2606.820068,2612.798096,2595.199951,2601.550049,834000,0,0.0,0
2021-07-08,2565.000000,2600.649902,2560.800049,2583.540039,989000,0,0.0,0


In [ ]:
import numpy as np

In [ ]:
X = []

In [ ]:
for i in  range(4252-12):
  x = df.iloc[i:i+10,0:5]
  #x["ID"] = range(1,11)
  X.append(x.to_numpy().ravel())

In [ ]:
X[0].shape

(50,)

In [ ]:
y = df.iloc[9:4249,7]

In [ ]:
len(X)

4240

In [ ]:
y

Date
2004-09-01    1
2004-09-02    0
2004-09-03    1
2004-09-07    1
2004-09-08    1
             ..
2021-06-29    1
2021-06-30    1
2021-07-01    1
2021-07-02    1
2021-07-06    0
Name: recommend, Length: 4240, dtype: int64

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
from tensorflow.keras.layers import LSTM

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
X_scaler = StandardScaler()

In [ ]:
# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()
# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=180, activation="relu",input_dim=50))
nn_model.add(tf.keras.layers.Dense(units=360, activation="relu"))
# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [ ]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(0.01), metrics=["accuracy"])

In [ ]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
100/100 [==============================] - 1s 3ms/step - loss: 0.7211 - accuracy: 0.5267
Epoch 2/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6891 - accuracy: 0.5487
Epoch 3/200
100/100 [==============================] - 0s 3ms/step - loss: 0.6889 - accuracy: 0.5497
Epoch 4/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6904 - accuracy: 0.5506
Epoch 5/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6935 - accuracy: 0.5403
Epoch 6/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6899 - accuracy: 0.5462
Epoch 7/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6944 - accuracy: 0.5465
Epoch 8/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6886 - accuracy: 0.5531
Epoch 9/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6933 - accuracy: 0.5456
Epoch 10/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6887 - accura

In [ ]:

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

34/34 - 0s - loss: 0.7897 - accuracy: 0.5462
Loss: 0.7897130846977234, Accuracy: 0.546226441860199
